In [1]:
import torch
import torch.nn as nn

In [2]:
from importlib import reload

In [3]:
import libmotivation
reload(libmotivation)
from libmotivation import *


In [4]:
user_list = get_users_with_choose_difficulty()

In [5]:
user_to_difficulty_items = {}
for user in user_list:
  user_to_difficulty_items[user] = get_choose_difficulty_items_for_user(user)

In [6]:
for user in user_list:
  difficulty_items = user_to_difficulty_items[user]
  if len(difficulty_items) > 0:
    print(difficulty_items[0])
    break


{'_id': ObjectId('5c104667c6466e0014e3a47f'), 'url': 'https://www.facebook.com/', 'tab_id': 279, 'session_id': 0, 'is_new_session': True, 'is_preview_mode': False, 'is_suggestion_mode': False, 'is_previously_seen': False, 'type': 'impression', 'intervention': 'internal/choose_difficulty', 'userid': '8d2c9eb27dee2dc85bca705b', 'install_id': '70870c04da0d5fee89b69c40', 'day': 1075, 'synced': 0, 'timestamp': 1544570471554.0, 'localtime': 'Tue Dec 11 2018 23:57:49 GMT+0100 (Central European Standard Time)', 'itemid': 'a8b4d51da34a71ebb1b33824', 'log_major_ver': '8', 'log_minor_ver': '1', 'habitlab_version': '1.0.258', 'id': 1, 'logname': 'internal/choose_difficulty', 'timestamp_local': 1544569069091.0}


In [7]:
print(get_most_common_difficulty_for_user('8d2c9eb27dee2dc85bca705b'))

nothing


In [8]:
print(get_most_common_difficulty_overall())

nothing


In [9]:
def get_correctness_of_naive_overall_strategy(user):
  output = {'correct': 0, 'incorrect': 0}
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  most_common = get_most_common_difficulty_overall()
  if most_common == None:
    return None
  output['total'] = sum(difficulty_counts.values())
  if output['total'] == 0:
    return None
  output['correct'] = difficulty_counts.get(most_common, 0)
  output['incorrect'] = output['total'] - output['correct']
  output['accuracy'] = output['correct'] / output['total']
  return output

In [10]:
def get_correctness_of_naive_peruser_strategy(user):
  output = {'correct': 0, 'incorrect': 0}
  difficulty_counts = get_choose_difficulty_counts_for_user(user)
  most_common = get_most_common_difficulty_for_user(user)
  if most_common == None:
    return None
  output['correct'] = difficulty_counts[most_common]
  output['total'] = sum(difficulty_counts.values())
  output['incorrect'] = output['total'] - output['correct']
  output['accuracy'] = output['correct'] / output['total']
  return output

In [11]:
correctnes_of_naive_per_user = []
for user in user_list:
  correctness_info = get_correctness_of_naive_peruser_strategy(user)
  if correctness_info == None:
    continue
  accuracy = correctness_info['accuracy']
  correctnes_of_naive_per_user.append(accuracy)

print(np.median(correctnes_of_naive_per_user))
print(np.mean(correctnes_of_naive_per_user))

0.6666666666666666
0.7035437782954957


In [12]:
correctnes_of_naive_per_user = []
for user in user_list:
  correctness_info = get_correctness_of_naive_overall_strategy(user)
  if correctness_info == None:
    continue
  accuracy = correctness_info['accuracy']
  correctnes_of_naive_per_user.append(accuracy)

print(np.median(correctnes_of_naive_per_user))
print(np.mean(correctnes_of_naive_per_user))

0.3333333333333333
0.4041840943289177


In [ ]:
def make_tensors_for_user(user):
  return

def make_features_for_user():
  return

In [15]:
n_features = 4 # the choice at each step. nothing, easy, medium, hard
n_categories = 4 # the number of categories we are categorizing among. nothing, easy, medium, hard

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

n_hidden = 128
rnn = RNN(n_features, n_hidden, n_categories).cuda()